In [1]:
import numpy as np
import pandas as pd
import os
import math as m
import matplotlib.pyplot as plt
import time

## Import Data Functions

In [2]:
df = pd.read_csv(r"..\data\dataset_new_score_v3.csv").drop(columns="Unnamed: 0")
df.columns

Index(['titre', 'budget', 'recette', 'duree', 'realisateur', 'casting',
       'compagnies_production', 'suite', 'Comedy', 'Drama', 'Horror',
       'Thriller', 'Action', 'Adventure', 'Science Fiction', 'Fantasy',
       'Family', 'Crime', 'Romance', 'Animation', 'Mystery', 'War', 'Music',
       'Western', 'History', 'Documentary', 'TV Movie', 'benefice global',
       'saison', 'casting_score_sum', 'casting_score_mean',
       'casting_score_med', 'casting_score_min', 'casting_score_max',
       'director_score_sum', 'director_score_mean', 'director_score_med',
       'director_score_min', 'director_score_max', 'compagnies_score_sum',
       'compagnies_score_mean', 'compagnies_score_med', 'compagnies_score_min',
       'compagnies_score_max'],
      dtype='object')

## Datasets and Scaling

In [3]:
from sklearn.model_selection import train_test_split

TEST_RATIO = 0.9
 
X_train, X_test, Y_train, Y_test = train_test_split(df.drop(columns=["titre", "recette", "realisateur", "casting", "compagnies_production", "benefice global"]), df["benefice global"], test_size = TEST_RATIO, shuffle=True, random_state = 1)

In [4]:
from sklearn.preprocessing import StandardScaler

X_train_f = StandardScaler().fit(np.array(X_train)).transform(np.array(X_train))


X_test_f = StandardScaler().fit(np.array(X_test)).transform(np.array(X_test))

print("Done")

Done


## Model

In [5]:
from sklearn.ensemble import RandomForestRegressor

start = time.time()
regr = RandomForestRegressor(random_state=0)
regr.fit(X_train_f, Y_train)
end = time.time()
elapsed1 = end - start

print("Temps exécution : ", elapsed1)


Temps exécution :  0.59100341796875


In [6]:
from sklearn.metrics import mean_absolute_percentage_error 
from sklearn.metrics import mean_absolute_error

#Get the time of execution
start = time.time()


score_mape = mean_absolute_percentage_error(Y_test, regr.predict(X_test_f))
score_mae = mean_absolute_error(Y_test, regr.predict(X_test_f))


end = time.time()
elapsed2 = elapsed1 + end - start

print("Temps exécution : ", elapsed2)
print("MAPE : ", score_mape)
print("MAE : ", score_mae)

Temps exécution :  0.7312188148498535
MAPE :  7369951363425319.0
MAE :  295.4945200906402


Cross validation

In [7]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error 
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

#Get the time of execution

scorer = make_scorer(mean_absolute_error)
score = cross_val_score(regr, X_train_f, Y_train, cv=10, scoring=scorer)


print("%0.2f MAE with a standard deviation of %0.2f" % (score.mean(), score.std()))




scorer = make_scorer(mean_absolute_percentage_error)
score = cross_val_score(regr, X_train_f, Y_train, cv=10, scoring=scorer)



print("%0.2f MAPE with a standard deviation of %0.2f" % (score.mean(), score.std()))


251.26 MAE with a standard deviation of 70.10
4427401522786901.00 MAPE with a standard deviation of 5900621190227134.00


## Tuning some shit

In [20]:
K = 10 #crossvalidation
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error 
from sklearn.metrics import make_scorer

# Paramètres à piper so good
scorer = make_scorer(mean_absolute_percentage_error)
parameters = [{'max_depth': [k for k in range(3,10)], 'criterion': ["squared_error","friedman_mse","absolute_error","poisson"]}]
print("ça tune de la thune")
regr_t = GridSearchCV(DecisionTreeRegressor(), parameters, cv = K, scoring = scorer)
regr_t.fit(X_test_f, Y_test_f)

# On check les scores
print("Grid scores")
means = regr_t.cv_results_['mean_test_score']
stds = regr_t.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, regr_t.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

ça tune de la thune


c:\Users\kerac\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
35 fits failed out of a total of 140.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\kerac\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\kerac\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py", line 1342, in fit
    super().fit(
  File "c:\Users\kerac\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py"

Grid scores
0.672 (+/-1.525) for {'criterion': 'squared_error', 'max_depth': 3}
0.611 (+/-1.179) for {'criterion': 'squared_error', 'max_depth': 4}
0.622 (+/-1.251) for {'criterion': 'squared_error', 'max_depth': 5}
0.590 (+/-1.247) for {'criterion': 'squared_error', 'max_depth': 6}
0.593 (+/-1.217) for {'criterion': 'squared_error', 'max_depth': 7}
0.615 (+/-1.245) for {'criterion': 'squared_error', 'max_depth': 8}
0.612 (+/-1.259) for {'criterion': 'squared_error', 'max_depth': 9}
0.672 (+/-1.525) for {'criterion': 'friedman_mse', 'max_depth': 3}
0.611 (+/-1.179) for {'criterion': 'friedman_mse', 'max_depth': 4}
0.599 (+/-1.268) for {'criterion': 'friedman_mse', 'max_depth': 5}
0.618 (+/-1.211) for {'criterion': 'friedman_mse', 'max_depth': 6}
0.575 (+/-1.206) for {'criterion': 'friedman_mse', 'max_depth': 7}
0.606 (+/-1.204) for {'criterion': 'friedman_mse', 'max_depth': 8}
0.575 (+/-1.239) for {'criterion': 'friedman_mse', 'max_depth': 9}
0.824 (+/-1.562) for {'criterion': 'absolut